In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import f
import statsmodels.stats.stattools as stattools
from scipy.stats import mstats
from scipy.stats import zscore
from scipy.stats import shapiro


# Lendo a planilha com os dados 

In [4]:
def ler_planilha (path):
    df = pd.read_excel(path, sheet_name='Adequação da curva',usecols="M:S", skiprows=3, nrows=7)
    df = df.set_index("Replicatas") #df.set_index("Replicatas", inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')# Converter as colunas para formato numérico, se necessário
    df = df.round(2)
    return df

In [5]:
df  =  ler_planilha ('Cu_estudo_da_curva _de_50_a_20.xlsm')
df

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
Replicatas,,,,,,
1,73439.33,114422.95,137847.37,169874.87,219072.22,281650.94
2,74709.57,114855.20,136493.41,173125.94,222644.05,276065.01
3,75383.99,116134.25,135820.57,173186.48,222438.49,277585.04
4,76085.28,117401.73,136135.26,175045.79,223538.30,278630.83
5,76761.00,117268.61,137848.39,176333.11,220192.94,281322.57
6,76516.32,117733.62,138811.59,176232.38,220294.16,280352.98
7,76067.53,115657.58,138185.08,176470.95,220751.10,281291.21


# Teste Grubbs - Detecção de Outliers

In [20]:
df  =  ler_planilha ('Cu_estudo_da_curva _de_50_a_20.xlsm')
n = len(df)

# Calcular estatísticas
medias = df.mean().round(2)
desvios_padrao = df.std().round(2)
minimos = df.min().round(2)
maximos = df.max().round(2)

# Calcular os valores Gmin, Gmax e G
Gmin = ((medias - minimos) / desvios_padrao).round(4)
Gmax = ((maximos - medias) / desvios_padrao).round(4)

# Calcular o valor crítico
n = len(df) - 1  # graus de liberdade
alpha = 0.05 # significância do teste
#Aqui, usamos a função ppf da biblioteca SciPy para calcular o valor crítico usando a distribuição t de Student. 
#calcula a probabilidade de um evento ocorrer, considerando o nível de significância.
#Dividir alpha por 2 é uma correção porque estamos lidando com um teste bicaudal
#(testando tanto para valores excessivamente altos quanto para valores excessivamente baixos).
critical_value = (stats.t.ppf(1 - alpha / (2 * n), n - 2)).round(4)

# Calcular G e Outlier para cada padrão
G_values = []
outliers = []

for i in range(len(df.columns)):
    G = Gmax[i] if maximos[i] > medias[i] else Gmin[i]
    G_values.append(G.round(4))
    outlier = 'Sim' if G > critical_value else 'Não'
    outliers.append(outlier)

# Criar DataFrame de estatísticas gerais
estatisticas_gerais = pd.DataFrame(
    [medias, desvios_padrao, minimos, maximos, Gmin, Gmax],
    index=['Média', 'Desvio Padrão', 'Mínimo', 'Máximo', 'Gmin', 'Gmax']
)

# Adicionar G e Outlier como linhas ao DataFrame
estatisticas_gerais.loc['G'] = G_values
estatisticas_gerais.loc['Outlier'] = outliers

# Concatenar DataFrames mantendo a orientação das colunas
df = pd.concat([df, estatisticas_gerais])

valor_critico_df= pd.DataFrame([critical_value], index=['Valor Crítico'])
df = pd.concat([df, valor_critico_df])
df = df.fillna("")
df.rename(columns={0: 'Estatísticas'}, inplace=True)
df


,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6,Estatísticas
1,73439.33,114422.95,137847.37,169874.87,219072.22,281650.94,
2,74709.57,114855.2,136493.41,173125.94,222644.05,276065.01,
3,75383.99,116134.25,135820.57,173186.48,222438.49,277585.04,
4,76085.28,117401.73,136135.26,175045.79,223538.3,278630.83,
5,76761.0,117268.61,137848.39,176333.11,220192.94,281322.57,
6,76516.32,117733.62,138811.59,176232.38,220294.16,280352.98,
7,76067.53,115657.58,138185.08,176470.95,220751.1,281291.21,
Média,75566.15,116210.56,137305.95,174324.22,221275.89,279556.94,
Desvio Padrão,1164.88,1303.98,1144.86,2426.69,1612.98,2163.46,
Mínimo,73439.33,114422.95,135820.57,169874.87,219072.22,276065.01,


# Teste Shapiro Wilk - Avaliação de normalidade dos resíduos

In [7]:
def shapiro_teste(col):
    stat, p = shapiro(col)
    if p > 0.05:
        return True
    else:
        return False

def shapiro_estatistica(col):
    stat, p = shapiro(col)
    return stat

def shapiro_p(col):
    stat, p = shapiro(col)
    return p

def shapiro_df(df):
    estatisticas_shapiro = df.apply(shapiro_estatistica)
    estatisticas_df = pd.DataFrame([estatisticas_shapiro], index=['Estatística W'])
    df = pd.concat([df, estatisticas_df])

    p_shapiro = df.apply(shapiro_p)
    p_df = pd.DataFrame([p_shapiro], index=['Valor p'])
    df = pd.concat([df, p_df])

    normal_shapiro = df.apply(shapiro_teste).apply(lambda x: "Sim" if x else "Não")
    normal_df = pd.DataFrame([normal_shapiro], index=['Distribuição Normal'])
    df = pd.concat([df, normal_df])
    return df

In [8]:
df  =  ler_planilha ('Cu_estudo_da_curva _de_50_a_20.xlsm')
resultados = shapiro_df(df)
resultados

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
1,73439.33,114422.95,137847.37,169874.87,219072.22,281650.94
2,74709.57,114855.2,136493.41,173125.94,222644.05,276065.01
3,75383.99,116134.25,135820.57,173186.48,222438.49,277585.04
4,76085.28,117401.73,136135.26,175045.79,223538.3,278630.83
5,76761.0,117268.61,137848.39,176333.11,220192.94,281322.57
6,76516.32,117733.62,138811.59,176232.38,220294.16,280352.98
7,76067.53,115657.58,138185.08,176470.95,220751.1,281291.21
Estatística W,0.906179,0.918947,0.910628,0.859808,0.938822,0.886165
Valor p,0.000003,0.000002,0.000002,0.000003,0.000002,0.000002
Distribuição Normal,Não,Não,Não,Não,Não,Não


In [9]:
#df  =  ler_planilha ('Testes_Estatisticos.xlsx')
#from scipy.stats import shapiro
#for col in df.columns:
#    stat, p = shapiro(df[col])
#    print(f'Coluna {col}:')
#    print('Estatística de teste:', stat)
#    print('Valor p:', p)
#    if p > 0.05:
#        print('Os dados parecem seguir uma distribuição normal.\n')
#    else:
#        print('Os dados não seguem uma distribuição normal.\n')

# Teste Cochran - Avaliação de Homecedasticidade

In [19]:
df  =  ler_planilha ('Cu_estudo_da_curva _de_50_a_20.xlsm')
df_inicial = df.copy()

# Calcula as variâncias de cada grupo
# O argumento opcional ddof indica o número de graus de liberdade a serem usados no cálculo da variância.
#Especificar ddof=1 significa que estamos calculando a variância amostral (usando N-1 como denominador, onde N é o número de observações).
#Isso é comumente usado em estatísticas para ajustar a variância amostral para um tamanho de amostra menor.
variances = df.var(ddof=1)

variances_df = pd.DataFrame([variances], index=['Variância'])
df = pd.concat([df, variances_df])

# Identifica a variância máxima e mínima
max_variance = variances.max()
variance_max_df= pd.DataFrame([max_variance], index=['Variância Max'])
df = pd.concat([df, variance_max_df])

min_variance = variances.min()
variance_min_df= pd.DataFrame([min_variance], index=['Variância Min'])
df = pd.concat([df, variance_min_df])

soma_variances = df.loc['Variância'].sum()
soma_variances_df= pd.DataFrame([soma_variances], index=['Soma das Variâncias'])
df = pd.concat([df, soma_variances_df])

# Calcula a estatística de teste Cochrane
sum_variances = variances.sum()
cochrane_statistic = max_variance / sum_variances # Ccalc =  Variância Max/Soma das Variâncias
#cochrane_statistic = max_variance / min_variance  # Ccalc =  Variância Max/Variância Min
cochrane_statistic_df= pd.DataFrame([cochrane_statistic], index=['Cochrane Estatistica'])
df = pd.concat([df, cochrane_statistic_df])

# Graus de liberdade para o numerador e denominador da distribuição F
df_num = df_inicial.shape[0] - 1  # Graus de liberdade do numerador (entre grupos)
df_den = df_inicial.size - df.shape[0]  # Graus de liberdade do denominador (dentro dos grupos)

# Valor crítico com 95% de confiança (alpha = 0.05)
valor_critico = f.ppf(1 - 0.05, df_num, df_den)
valor_critico_df= pd.DataFrame([valor_critico], index=['Cochrane Tabelado'])
df = pd.concat([df, valor_critico_df])

if cochrane_statistic < valor_critico:
    Resultado = "HOMOCEDÁSTICO"
    Resultado_df= pd.DataFrame([Resultado], index=['Resultado'])
    df = pd.concat([df, Resultado_df])
else:
    Resultado = "Não HOMOCEDÁSTICO"
    Resultado_df= pd.DataFrame([Resultado], index=['Resultado'])
    df = pd.concat([df, Resultado_df])

df = df.fillna("")
df.rename(columns={0: 'Estatísticas'}, inplace=True)
df



,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6,Estatísticas
1,73439.33,114422.95,137847.37,169874.87,219072.22,281650.94,
2,74709.57,114855.2,136493.41,173125.94,222644.05,276065.01,
3,75383.99,116134.25,135820.57,173186.48,222438.49,277585.04,
4,76085.28,117401.73,136135.26,175045.79,223538.3,278630.83,
5,76761.0,117268.61,137848.39,176333.11,220192.94,281322.57,
6,76516.32,117733.62,138811.59,176232.38,220294.16,280352.98,
7,76067.53,115657.58,138185.08,176470.95,220751.1,281291.21,
Variância,1356940.381162,1700371.31219,1310714.701424,5888811.54399,2601690.010329,4680550.313067,
Variância Max,,,,,,,5888811.54399
Variância Min,,,,,,,1310714.701424


# Teste Anova

In [11]:
df  =  ler_planilha ('Cu_estudo_da_curva _de_50_a_20.xlsm')

# ANOVA (falta de ajuste e erro puro)
F, pvalue = stats.f_oneway(df['Padrão 1'], df['Padrão 1'])
if pvalue < 0.05:
    print('Existe uma diferença significativa entre os grupos.')
else:
    print('Não existe uma diferença significativa entre os grupos.')

Não existe uma diferença significativa entre os grupos.


# Durbin Watson

In [12]:
df  =  ler_planilha ('Cu_estudo_da_curva _de_50_a_20.xlsm')

# Teste de independência dos resíduos (Durbin Watson)
# Obter o teste de Durbin-Watson
dw = stattools.durbin_watson(df['Padrão 1'])

# Imprimir o resultado do teste
print(dw)

8.199227618926422e-05
